In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import odl
from odl.contrib import tomo

space = odl.uniform_discr([-112,-112],[112,112], 128)

phant = phant = odl.phantom.transmission.shepp_logan(space, modified = False)
phant.show()

max_it = 5000
 


In [ ]:
# Construct operator

geo = odl.tomo.cone_beam_geometry(space,780, 220, num_angles=128, short_scan=True)

A = odl.tomo.operators.ray_trafo.RayTransform(space, geo, impl='astra_cuda')

y = A(phant)
y.show()

print("Suppressor")

In [ ]:
# Make noisy data

gaussian = odl.phantom.noise.white_noise(A.range, stddev=0.005)
y_n = y + gaussian * y.norm() / (100 * gaussian.norm())

y_n.show()

print("Suppressor")

In [ ]:
# Construct Huber functional

# Construct gradient
x_grad = odl.discr.diff_ops.Gradient(space)

eps = 0.1
alpha = 0.1

huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

# Combine into variational model

norm_y =  odl.solvers.functional.default_functionals.L2Norm(A.range) 
norm_y2 =  odl.solvers.functional.default_functionals.L2NormSquared(A.range) 

data_fidel = norm_y2.translated(y_n)*A


var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

In [ ]:
# Minimize variational model using, what?

x = space.zero()
odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)

In [ ]:
x.show()

print("Relative difference to true  y: ", (A(x)- y).norm()/y.norm())
print("Relative difference to noisy y: ", (A(x)- y_n).norm()/y.norm())
print("Relative difference in x: ", (x-phant).norm()/phant.norm())

In [ ]:
# Looper trying different eps, rough search

eps_list = 2**np.linspace(-5,0,8)
x_grad = odl.discr.diff_ops.Gradient(space)

alpha = 100

norm_y =  odl.solvers.functional.default_functionals.L2Norm(A.range) 

data_fidel = norm_y.translated(y_n)*A

y_bad = []
x_bad = []
x_norm = []

for eps in eps_list:
    huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())




In [ ]:
plt.figure()
plt.loglog(y_bad,x_norm)
plt.show()

plt.figure()
plt.semilogy(eps_list,y_bad)
plt.show()

plt.figure()
plt.semilogy(eps_list,x_bad)
plt.show()

eps_min = eps_list[np.argmin(y_bad)]
print("Best with eps = ", eps_min)

In [ ]:
eps_range = np.linspace(eps_min,eps_min*4, 8)

# Fine search

y_bad = []
x_bad = []
x_norm = []

for eps in eps_range:
    huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())



In [ ]:
eps_min = eps_range[np.argmin(y_bad)]
print("Best with eps = ", eps_min)

In [ ]:
plt.figure()
plt.loglog(y_bad,x_norm)
plt.show()

plt.figure()
plt.semilogy(eps_range,y_bad)
plt.show()

plt.figure()
plt.semilogy(eps_range,x_bad)
plt.show()

In [ ]:
eps = eps_min
# Lets search for lambda as well
huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

alp_list = 2**(np.linspace(3,10,10))

y_bad = []
x_bad = []
x_norm = []

for alpha in alp_list:

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())

In [ ]:
plt.figure()
plt.loglog(y_bad,x_norm)
plt.show()

plt.figure()
plt.semilogy(alp_list,y_bad)
plt.show()

plt.figure()
plt.semilogy(alp_list,x_bad)
plt.show()

alp_min = alp_list[np.argmin(y_bad)]
print("Best with alpha = ", alp_min)

In [ ]:
# Fine search for alpa

alp_list = np.linspace(alp_min/2, alp_min*2,10)

y_bad = []
x_bad = []
x_norm = []

for alpha in alp_list:

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())

In [ ]:
plt.figure()
plt.loglog(y_bad,x_norm)
plt.show()

plt.figure()
plt.semilogy(alp_list,y_bad)
plt.show()

plt.figure()
plt.semilogy(alp_list,x_bad)
plt.show()

alp_min = alp_list[np.argmin(y_bad)]
print("Best with alpha = ", alp_min)

In [ ]:
# New search on eps

alpha = 146.3
eps_range = np.linspace(0.15*0.05,0.15*1.05, 10)

y_bad = []
x_bad = []
x_norm = []

for eps in eps_range:
    huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())


In [ ]:
eps_min = eps_range[np.argmin(y_bad)]
print("Best with eps = ", eps_min)

In [ ]:
plt.figure()
plt.semilogy(eps_range,y_bad)
plt.show()

plt.figure()
plt.semilogy(eps_range,x_bad)
plt.show()

In [ ]:
# final search allpha/lambda

eps = eps_min
huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

alp_list = np.linspace(alp_min*0.5,alp_min*5,10)

y_bad = []
x_bad = []
x_norm = []

for alpha in alp_list:

    # Combine into variational model
    var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

    # Minimize variational model using, what?
    x = space.zero()
    odl.solvers.smooth.gradient.adam(var_model, x, maxiter=max_it)
    y_bad.append((A(x)- y).norm()/y.norm())
    x_bad.append((x-phant).norm()/phant.norm())
    x_norm.append(x.norm())

In [ ]:
plt.figure()
plt.semilogy(alp_list,y_bad)
plt.show()

plt.figure()
plt.semilogy(alp_list,x_bad)
plt.show()

alp_min = alp_list[np.argmin(y_bad)]
print("Best with alpha = ", alp_min)

In [ ]:
# Final reconstruction

alpha = alp_min 
eps = eps_min

huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

x = space.zero()
odl.solvers.smooth.gradient.adam(var_model, x, maxiter = 20000)

In [ ]:
x.show()

print("Relative difference to true  y: ", (A(x)- y).norm()/y.norm())
print("Relative difference to noisy y: ", (A(x)- y_n).norm()/y.norm())
print("Relative difference in x: ", (x-phant).norm()/phant.norm())



In [ ]:
alpha = alp_min 
eps = 0.01

huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

x = space.zero()
odl.solvers.smooth.gradient.adam(var_model, x, maxiter = 20000)

In [ ]:
x.show()

print("Relative difference to true  y: ", (A(x)- y).norm()/y.norm())
print("Relative difference to noisy y: ", (A(x)- y_n).norm()/y.norm())
print("Relative difference in x: ", (x-phant).norm()/phant.norm())

In [ ]:
alpha = alp_min
eps = 0.0005

huber_functional = odl.solvers.functional.default_functionals.Huber(x_grad.range, gamma = eps)

var_model = 0.5 * odl.solvers.FunctionalProduct(data_fidel,data_fidel) + alpha * huber_functional*x_grad

x = space.zero()
odl.solvers.smooth.gradient.adam(var_model, x, maxiter = 20000)

In [ ]:
x.show()

print("Relative difference to true  y: ", (A(x)- y).norm()/y.norm())
print("Relative difference to noisy y: ", (A(x)- y_n).norm()/y.norm())
print("Relative difference in x: ", (x-phant).norm()/phant.norm())

In [ ]:
print(alpha)